In [ ]:
#imports for website dev/plotting
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px

#imports for data + manipulation
import pandas as pd
import numpy as np
import imdb
from bs4 import BeautifulSoup
import requests
import re

#other imports
import os

## Get Data
Using the imdbpy API to get data about Rachel Weisz from IMDB.

In [ ]:
ia = imdb.IMDb()

In [ ]:
movie_cols = ia.get_movie_infoset()
star_cols = ia.get_person_infoset()

In [ ]:
rachel_id = ia.search_person('Rachel Weisz')[0].personID

In [ ]:
rachel = ia.get_person(rachel_id)

In [ ]:
rachel_movies = rachel['filmography'][0]['actress']
rachel_movie_ids = [i.movieID for i in rachel_movies]

## Data Cleaning
Pulling relevant data from each movie from Rachel Weisz's filmography. Putting the data into a DataFrame to be saved to a csv so that it can be easily accessed in the Dash app. Made function to scrape boxofficemojo to get the box office gross of each movie.

In [ ]:
#mov.infoset2keys (to see the info pulled)
#var(mov) (to see the attributes of Movies/Person)
movies = []
important_keys = ['original title', 'genres', 'runtimes', 'countries', 'country codes', 'votes',
                 'year', 'rating', 'original air date', 'kind', 'awards']

In [ ]:
for i in rachel_movie_ids:
    movie = ia.get_movie(i, info = ['main', 'awards'])
    movie_info = []
    for k in important_keys:
        try:
            val = movie[k]
            movie_info.append(val)
        except:
            movie_info.append(np.nan)
    
    movies.append(movie_info)

In [ ]:
df = pd.DataFrame(movies, columns = important_keys)
df['id'] = rachel_movie_ids
df.head()

In [ ]:
def box_office(movie_id):
    try:
        imdb_id = 'tt' + str(movie_id)

        # Fetch the box office numbers
        base = 'https://www.boxofficemojo.com'
        url = base + '/title/' + imdb_id
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')
        table = soup('th', text=re.compile(r'Release Group'))[0].parent.parent
        group = table.findAll('tr', recursive=False)[1].find('a').get('href')
        url = base + group

        # Get total earnings domestic and international
        source = requests.get(url).text
        soup = BeautifulSoup(source,'lxml')
        earnings = soup('h2', text=re.compile(r'Rollout'))[0].parent.parent.findAll('div')
        domestic = earnings[1].find('span', {'class': 'money'}).get_text()
        domestic_url = earnings[1].find('a').get('href')
        international = earnings[2].find('span', {'class': 'money'}).get_text()

        domestic = domestic.replace('$', '').replace(',', '')
        international = international.strip('$').replace(',', '')
    except:
        domestic = np.nan
        international = np.nan
    
    return str(domestic) + ',' + str(international)

In [ ]:
gross = df.loc[df['kind'] == 'movie']['id'].apply(box_office)

In [ ]:
gross_re = gross.str.split(',', expand=True)
gross_re.columns = ['domestic', 'international']
gross_re = gross_re.replace('nan', np.nan)
gross_re['domestic'] = gross_re['domestic'].astype(float)
gross_re['international'] = gross_re['international'].astype(float)

In [ ]:
rachel_df = pd.concat([df.loc[df['kind'] == 'movie'], gross_re], axis = 1)

In [ ]:
rachel_df['runtimes'] = rachel_df['runtimes'].apply(lambda x: x[0] if type(x) != float else x)
rachel_df['total'] = rachel_df['domestic'] + rachel_df['international']

In [ ]:
rachel_df.reset_index(drop = True).to_csv('rachel.csv')

In [ ]:
countries = rachel_df.groupby('year')['countries'].sum()
countries = countries.reset_index()
yearly = countries['countries'].apply(lambda x: pd.Series(x).value_counts()).fillna(0)
yearly['year'] = countries.year
country_df = pd.melt(yearly, 'year').rename({'variable': 'Country', 'value': 'Movie Count', 'year': 'Year'}, axis = 1)
country_df = country_df.loc[country_df['Movie Count'] > 0].sort_values('Year').reset_index(drop = True)

In [ ]:
px.choropleth(country_df, locations =  'Country', locationmode = 'country names', color = 'Movie Count', 
             animation_frame="Year")

In [ ]:
#TODO: network graph, something about financials and being gay

In [ ]:
dropped = rachel_df.dropna(subset = ['domestic', 'international'])\
[['original title', 'domestic', 'international', 'total']]

labels = dropped['original title'].tolist()
parents = ['Rachel Weisz Movies' for x in labels]

labels.append('Rachel Weisz Movies')
parents.append('')

fig = go.Figure()
fig.add_trace(go.Treemap(
    labels = labels,
    parents = parents,
    ))
fig.show()

In [ ]:
import plotly
plotly.__version__